In [61]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [62]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Butaritari,KI,2022-07-13 17:17:16,3.0707,172.7902,82.36,78,88,18.86
1,1,Bredasdorp,ZA,2022-07-13 17:17:16,-34.5322,20.0403,58.42,64,43,18.72
2,2,Kavaratti,IN,2022-07-13 17:17:17,10.5669,72.6420,81.36,83,99,19.15
3,3,Vaini,TO,2022-07-13 17:17:17,-21.2000,-175.2000,75.36,94,75,12.66
4,4,Ushuaia,AR,2022-07-13 17:17:17,-54.8000,-68.3000,38.82,75,75,10.36


In [63]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [64]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [65]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [66]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [67]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [68]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [69]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [70]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
26,26,Bilma,NE,2022-07-13 17:17:24,18.6853,12.9164,103.14,10,16,11.21
35,35,Cabra,ES,2022-07-13 17:17:27,37.4725,-4.4421,99.12,14,4,16.24
40,40,Riyadh,SA,2022-07-13 17:15:10,24.6877,46.7219,100.54,8,89,9.40
93,93,Maine-Soroa,NE,2022-07-13 17:19:42,13.2114,12.0241,99.32,25,24,11.14
260,260,Perigueux,FR,2022-07-13 17:23:30,45.1833,0.7167,101.08,13,4,5.88
454,454,Buraydah,SA,2022-07-13 17:28:26,26.3260,43.9750,102.65,11,100,11.65
490,490,Banda,IN,2022-07-13 17:29:36,25.4833,80.3333,100.18,33,74,15.97


In [71]:
preferred_cities_df.count()

City_ID       7
City          7
Country       7
Date          7
Lat           7
Lng           7
Max Temp      7
Humidity      7
Cloudiness    7
Wind Speed    7
dtype: int64

In [72]:
#Continue with lesson 6.5.4

In [73]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
26,Bilma,NE,103.14,18.6853,12.9164,
35,Cabra,ES,99.12,37.4725,-4.4421,
40,Riyadh,SA,100.54,24.6877,46.7219,
93,Maine-Soroa,NE,99.32,13.2114,12.0241,
260,Perigueux,FR,101.08,45.1833,0.7167,
454,Buraydah,SA,102.65,26.3260,43.9750,
490,Banda,IN,100.18,25.4833,80.3333,


In [74]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# Iterate through the DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


In [75]:
hotel_df 

,City,Country,Max Temp,Lat,Lng,Hotel Name
26,Bilma,NE,103.14,18.6853,12.9164,Expeditions Ténére voyages
35,Cabra,ES,99.12,37.4725,-4.4421,MS Fuente Las Piedras
40,Riyadh,SA,100.54,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"
93,Maine-Soroa,NE,99.32,13.2114,12.0241,Complexe
260,Perigueux,FR,101.08,45.1833,0.7167,Hôtel Mercure Périgueux Centre
454,Buraydah,SA,102.65,26.3260,43.9750,Mövenpick Qassim
490,Banda,IN,100.18,25.4833,80.3333,Hotel Comfort-Inn


In [76]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [77]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [78]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))